In [5]:
import pandas as pd
import numpy as np
from os import chdir
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
chdir("../Packages/") 
import pymorphy2
import pymorphy2_dicts

In [7]:
chdir("../Compile")
import Data_filter as DF
import Analyzier

In [86]:
TrainingData = pd.DataFrame.from_csv("../../Data/training_data.csv")

In [116]:
Data = TrainingData['sentence'].apply(DF.ParseSentence)
Data = Data.to_frame()
Data = Data.reset_index()
del(Data['index'])

In [117]:
Vectorizer = CountVectorizer(analyzer='word')

In [118]:
def ArrayToString(Array):
    Result = ""
    for word in Array:
        Result+=word+" "
    return Result

In [119]:
Data = Data.join(pd.DataFrame.from_dict(Vectorizer.fit_transform(Data['sentence'].apply(lambda x: ArrayToString(x[0]))).toarray()))

In [120]:
#Data['?'] = Data['sentence'].apply(lambda x: x[1]['?'])
#Data['!'] = Data['sentence'].apply(lambda x: x[1]['!'])
#Data['.'] = Data['sentence'].apply(lambda x: x[1]['.'])
#Data['sum'] = Data['sentence'].apply(lambda x: Analyzier.AnalyzeWords(x[0]))
#Data['negative'] = Data['sentence'].apply(lambda x: Analyzier.AnalyzeNegativeBigrams(x[0]))
#Data['positive'] = Data['sentence'].apply(lambda x: Analyzier.AnalyzePositiveBigrams(x[0]))
#Data['neutral'] = Data['sentence'].apply(lambda x: Analyzier.AnalyzeNeutralBigrams(x[0]))
del(Data['sentence'])

In [121]:
Data.head(3)

,0,1,2,3,4,5,6,7,8,9,...,2671,2672,2673,2674,2675,2676,2677,2678,2679,2680
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
Training = TrainingData['label'].values

In [123]:
model =  RandomForestClassifier(n_estimators=1000, n_jobs=-1)

In [127]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
Xtr,Xval,Ytr,Yval = train_test_split(Data,Training,test_size=0.3, random_state=128)

In [128]:
model.fit(Xtr, Ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [129]:
print(f1_score(Yval, model.predict(Xval), average='macro'))

0.43314585638


In [107]:
model.feature_importances_

array([  8.12734032e-06,   2.58498199e-05,   2.78882064e-06, ...,
         6.94457025e-02,   5.62118228e-02,   7.23772137e-02])

In [108]:
Input = open("../../Data/data.txt")
PredictionData = []
for line in Input.readlines():
    PredictionData.append(line)
del(PredictionData[0])
Input.close()

In [109]:
PredictionData = pd.DataFrame(PredictionData)
PredictionData.columns = ['sentence']

In [110]:
PredictionData['parsed'] = PredictionData['sentence'].apply(DF.ParseSentence)
del(PredictionData['sentence'])

In [111]:
PredictionData = PredictionData.join(pd.DataFrame.from_dict(Vectorizer.transform(PredictionData['parsed'].apply(lambda x: ArrayToString(x[0]))).toarray()))

In [112]:
#PredictionData['?'] = PredictionData['parsed'].apply(lambda x: x[1]['?'])
#PredictionData['!'] = PredictionData['parsed'].apply(lambda x: x[1]['!'])
#PredictionData['.'] = PredictionData['parsed'].apply(lambda x: x[1]['.'])
#PredictionData['sum'] = PredictionData['parsed'].apply(lambda x: Analyzier.AnalyzeWords(x[0]))
PredictionData['negative'] = PredictionData['parsed'].apply(lambda x: Analyzier.AnalyzeNegativeBigrams(x[0]))
PredictionData['positive'] = PredictionData['parsed'].apply(lambda x: Analyzier.AnalyzePositiveBigrams(x[0]))
PredictionData['neutral'] = PredictionData['parsed'].apply(lambda x: Analyzier.AnalyzeNeutralBigrams(x[0]))
del(PredictionData['parsed'])

In [113]:
Answer = model.predict(PredictionData)

In [114]:
Output = open('result.txt', 'w')
Output.write("\"label\""+"\n")

8

In [115]:
for i in Answer:
    Output.write("\""+str(i)+"\""+"\n")
Output.close()